<a href="https://colab.research.google.com/github/NKASG/Pneumonia-detection/blob/main/Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import models, layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_ds = image_dataset_from_directory('/content/drive/MyDrive/chest_xrayss/trains',shuffle=True, image_size=(256,256), batch_size=32 )

Found 5256 files belonging to 2 classes.


In [ ]:
test_ds = image_dataset_from_directory('/content/drive/MyDrive/chest_xrayss/test',shuffle=True, image_size=(256,256), batch_size=32 )

Found 624 files belonging to 2 classes.


In [ ]:
val_ds = image_dataset_from_directory('/content/drive/MyDrive/chest_xrayss/val',shuffle=True, image_size=(256,256), batch_size=32 )

Found 16 files belonging to 2 classes.


In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(256,256),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
input_shape = (32,256,256,3)

In [ ]:
    Model = keras.models.Sequential([
resize_and_rescale ,
    
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax'),

  
   
    
])
     
   

In [ ]:
 Model.build(input_shape)

In [ ]:
Model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
Model.summary()

In [ ]:
history = Model.fit( train_ds, batch_size=32, validation_data=val_ds, verbose=1, epochs=5)

Epoch 1/5
165/165 [==============================] - 999s 2s/step - loss: 0.5873 - accuracy: 0.7392 - val_loss: 0.8360 - val_accuracy: 0.5000
Epoch 2/5
165/165 [==============================] - 370s 2s/step - loss: 0.4994 - accuracy: 0.7544 - val_loss: 0.8052 - val_accuracy: 0.5625
Epoch 3/5
165/165 [==============================] - 378s 2s/step - loss: 0.3792 - accuracy: 0.8326 - val_loss: 0.5194 - val_accuracy: 0.7500
Epoch 4/5
165/165 [==============================] - 390s 2s/step - loss: 0.2620 - accuracy: 0.8950 - val_loss: 0.7702 - val_accuracy: 0.6250
Epoch 5/5
165/165 [==============================] - 379s 2s/step - loss: 0.2002 - accuracy: 0.9247 - val_loss: 0.8831 - val_accuracy: 0.6875


In [ ]:
scores = Model.evaluate(test_ds)